### Llamaindex 

It is a framework for building LLM applications. Langchain's main focus in to create complex chains using various components and create complex multi-step applications, while llamaindex is designed to manage and index large, unstructured data collections (like documents) to make them accessible and useful to LLMs.

It focuses on building retrieval-based systems by indexing data and enabling efficient querying over large document sets.

Llamaindex is an excellent choice for building RAG applications where the LLM queries indexed documents, and relevant context is retrieved and passed to the model.

In [33]:
%pip install -q llama-index faiss-cpu llama-index-embeddings-huggingface llama-parse llama-index-vector-stores-faiss

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

import os
from dotenv import load_dotenv

load_dotenv()

hugging_face_api_key = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

d:\VS Code\python\GenAI-Cookbook\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings = embed_model.get_text_embedding("Hello World!")

#### creates embedding of dimension size 384

In [7]:
len(embeddings) 

384

#### Read Documents

In [35]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

path = "../data/"

pdf_parser = SimpleDirectoryReader(input_dir=path, required_exts=['.pdf'])
documents = pdf_parser.load_data()

In [36]:
documents[0]

Document(id_='7c58139f-4432-40d2-92e1-bca110b85940', embedding=None, metadata={'page_label': 'I', 'file_name': 'who_stats_2022.pdf', 'file_path': 'd:\\VS Code\\python\\GenAI-Cookbook\\RAG_techniques\\..\\data\\who_stats_2022.pdf', 'file_type': 'application/pdf', 'file_size': 7925105, 'creation_date': '2024-09-08', 'last_modified_date': '2024-09-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='World \nHealth \nStatistics \n2022\nMonitoring  \nhealth for the  \nSDG s\nSustainable Development Goals', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [37]:
vars(documents[0])

{'id_': '7c58139f-4432-40d2-92e1-bca110b85940',
 'embedding': None,
 'metadata': {'page_label': 'I',
  'file_name': 'who_stats_2022.pdf',
  'file_path': 'd:\\VS Code\\python\\GenAI-Cookbook\\RAG_techniques\\..\\data\\who_stats_2022.pdf',
  'file_type': 'application/pdf',
  'file_size': 7925105,
  'creation_date': '2024-09-08',
  'last_modified_date': '2024-09-08'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'text': 'World \nHealth \nStatistics \n2022\nMonitoring  \nhealth for the  \nSDG s\nSustainable Development Goals',
 'mimetype': 'text/plain',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n'}

In [13]:
len(documents)

131

In [15]:
len(documents[100].text)

6373

In [55]:
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
import faiss

# create FAISS index with proper dimensions
faiss_index = faiss.IndexFlatL2(384) # pass the embed dimension
vector_store = FaissVectorStore(faiss_index=faiss_index)
print(vector_store)
storage_context = StorageContext.from_defaults(vector_store=vector_store) # context important as FAISS requires pkl file

stores_text=False is_embedding_query=True


In [ ]:
index = VectorStoreIndex.from_documents(documents=documents, embed_model=embed_model, show_progress=True)

### Retriever to get similar docs

In [116]:
retriever = index.as_retriever(similarity_top_k=7)

In [57]:
question = "What was impact of covid-19 on middle eastern countries?"

In [58]:
contexts = retriever.retrieve(question)

In [59]:
contexts

[NodeWithScore(node=TextNode(id_='35ae21be-25bc-4471-b170-6f31070ac537', embedding=None, metadata={'page_label': '77', 'file_name': 'who_stats_2022.pdf', 'file_path': 'd:\\VS Code\\python\\GenAI-Cookbook\\RAG_techniques\\..\\data\\who_stats_2022.pdf', 'file_type': 'application/pdf', 'file_size': 7925105, 'creation_date': '2024-09-08', 'last_modified_date': '2024-09-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eb94dc14-4746-48ab-9233-abd89dd9f97e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '77', 'file_name': 'who_stats_2022.pdf', 'file_path': 'd:\\VS Code\\python\\GenAI-Cookbook\\RAG_techniques\\..\\data\\who_stats_2022.pdf', 'file_type': 'application/pdf', 'file_size': 7925105, 

#### Each context document has a score

In [74]:
contexts[0].score

0.7730778862568785

#### Extract just the text from node document

In [60]:
context_list = [n.get_content() for n in contexts]
len(context_list)

10

In [63]:
%pip install -q llama-index-llms-gemini google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [64]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7, max_retries=3)

In [68]:
%pip install -q llama-index-llms-langchain

Note: you may need to restart the kernel to use updated packages.


In [133]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=10)

In [136]:
response = query_engine.query(question)

In [137]:
print(response.response)

The COVID-19 pandemic highlighted the need for more frequent data to monitor sudden changes in mortality in the Eastern Mediterranean Region. Several countries in the region were able to establish rapid mortality surveillance systems and collect data by age and sex.  



In [105]:
vars(response)

{'response': 'The COVID-19 pandemic had a significant impact on the Eastern Mediterranean Region, particularly in terms of financial protection. While service coverage increased, catastrophic health spending also increased, indicating a worsening of financial protection. \n',
 'source_nodes': [NodeWithScore(node=TextNode(id_='35ae21be-25bc-4471-b170-6f31070ac537', embedding=None, metadata={'page_label': '77', 'file_name': 'who_stats_2022.pdf', 'file_path': 'd:\\VS Code\\python\\GenAI-Cookbook\\RAG_techniques\\..\\data\\who_stats_2022.pdf', 'file_type': 'application/pdf', 'file_size': 7925105, 'creation_date': '2024-09-08', 'last_modified_date': '2024-09-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='

### Query with prompt template

In [75]:
from llama_index.core import PromptTemplate

from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

In [117]:
def getContextDocuments(query:str):
    contexts = retriever.retrieve(question)
    
    context_list = [n.get_content() for n in contexts]
    len(context_list)
    
    return  "\n\n".join(str(x) for x in context_list)

### Print template filled with query and context

In [118]:
print(qa_template.format(query_str= question, context_str= getContextDocuments(question)))

We have provided context information below. 
---------------------
82•European Region
Disruptions of essential health services
1  Responses were obtained from 23 of 53 WHO European Region Member States.COVID-19 has added pressure to all three pillars of UHC: access to health services, quality of care, and financial protection. In 2020, the results of the first WHO pulse survey revealed that over 40% of essential health services in the WHO European region had been at least partially disrupted (4), with the disruptions occurring across all aspects of health systems. Results from the second WHO pulse survey showed that substantial disruptions persisted in 2021, with around 29% of services at least partially disrupted in the first three months of 2021 (5).1 During the rest of 2021, the extent and magnitude of those disruptions decreased compared to 2020 (6). The main causes of the disruptions have also shifted. On the supply side, shortages of personal protective equipment seemed to become

In [95]:
qengine = index.as_query_engine(llm=llm, text_qa_template=qa_template)

In [96]:
qengine.query(question)

Response(response="The provided text focuses on the impact of COVID-19 on the European and South-East Asia regions. It doesn't contain any information about the Middle East. \n", source_nodes=[NodeWithScore(node=TextNode(id_='35ae21be-25bc-4471-b170-6f31070ac537', embedding=None, metadata={'page_label': '77', 'file_name': 'who_stats_2022.pdf', 'file_path': 'd:\\VS Code\\python\\GenAI-Cookbook\\RAG_techniques\\..\\data\\who_stats_2022.pdf', 'file_type': 'application/pdf', 'file_size': 7925105, 'creation_date': '2024-09-08', 'last_modified_date': '2024-09-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eb94dc14-4746-48ab-9233-abd89dd9f97e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '

In [119]:
final_prompt = qa_template.format(query_str= question, context_str= getContextDocuments(question))

In [120]:
len(final_prompt)

22017

### Update prompt template as this new template

In [125]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [128]:
from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

### Display the prompt template

In [129]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

We have provided context information below. 
---------------------
{context_str}
---------------------
Given this information, please answer the question: {query_str}



<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

### Input question to query which will find similar k docs and fill as context

In [134]:
llm_response = query_engine.query(question)

In [135]:
print(llm_response.response)

The COVID-19 pandemic highlighted the need for more frequent mortality data in the Eastern Mediterranean Region.  Countries in the region worked with the WHO to improve their mortality registration systems, and some were able to establish rapid mortality surveillance. 

